<a href="https://colab.research.google.com/github/hikaruy0804/DeepLearning/blob/main/2_2_%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%EF%BC%88%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF_GPU%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[0] 畳み込み層をGPU上で学習する方法
===============
## GPUに変更
ランタイム→ランタイムのタイプを変更→ハードウェアアクセラレータ→T4 GPU→保存
テンソルをGPUに転送するのと同じように、ニューラルネットワークをGPUに転送します。

<br>

**画像分類器の訓練**

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# GPU搭載のCUDA環境を前提としており、その場合はcuda:0と出力されるはずです
print(device)

2-2.深層学習（畳み込み-GPU）

以下の手順に従って実施します：

1. ``torchvision``でCIFAR10の訓練データとテストデータの読み込みと正規化
2. 畳み込みニューラルネットワークの定義
3. 損失関数の定義
4. 訓練データを用いたモデルの訓練
5. テストデータでモデルをテスト


[1] CIFAR10の読み込みと正規化
========

In [ ]:
%matplotlib inline

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 画像の表示関数


def imshow(img):
    img = img / 2 + 0.5     # 正規化を戻す
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# 適当な訓練セットの画像を取得
dataiter = iter(trainloader)
images, labels = next(dataiter)

# 画像の表示
imshow(torchvision.utils.make_grid(images))
# ラベルの表示
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

[2] 畳み込みニューラルネットワークの定義
=======


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #入力チャンネル、出力チャンネル、カーネルサイズ
        self.pool = nn.MaxPool2d(2, 2) #プーリングサイズ
        self.conv2 = nn.Conv2d(6, 16, 5) #入力チャンネル、出力チャンネル、カーネルサイズ
        self.fc1 = nn.Linear(16 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

全モジュールのパラメータとバッファをCUDAテンソルに変換します。

In [ ]:
net.to(device)
# inputs, labels = data[0].to(device), data[1].to(device)

[3] 損失関数とオプティマイザの定義
=======


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

[4] ネットワークの訓練
=======
モデルのサイズが小さいのでCPUで実行した際と比較して、大幅な高速化にはならないかと思います。

In [ ]:
# 日本語訳注：GPU版で訓練を実行した場合

# optimizerを再定義（netがGPU上に移動したので）
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # データセットを何巡繰り返すか

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        # 入力を取得します; 変数dataはリスト[inputs, labels]です
        inputs, labels = data[0].to(device), data[1].to(device)

        # 勾配を0に初期化
        optimizer.zero_grad()

        # 順伝搬、逆伝搬、パラメータ更新
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 統計情報を出力
        running_loss += loss.item()
        if i % 2000 == 1999:    #  2000ミニバッチごとに出力
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# 演習
## 試してみよう。

- モデルの層を深くする
- 以下のコードをモデル構築のコードと入れ替えて実行する

## 調べてみよう。
- [Google Colaboratoryのスペック](https://zenn.dev/kun432/scraps/ff58490b5133ce)




---




### Dropout（ドロップアウト）

ドロップアウトはニューラルネットワークの正則化手法の一つで、モデルの過学習を防ぐために使用されます。

#### 過学習とは？
過学習（オーバーフィッティング）は、モデルが訓練データにはうまく適応するものの、新しいデータ（テストデータ）にはうまく適応できない状態を指します。これは、モデルが訓練データのノイズや重要でないパターンまで学習してしまうことが原因です。

#### ドロップアウトの効果
1. **アンサンブル効果**: 学習中にランダムにニューロンを「無効化」します。これにより、ネットワークは複数の異なるモデルの平均のように機能し、一般化（汎化）能力が向上します。
2. **特徴の共適応抑制**: ドロップアウトは各ニューロンが他のニューロンに過度に依存しないようにします。これにより、ネットワークはより堅牢な特徴を学習し、一般化能力を向上させます。

#### ドロップアウトの使い方
1. **ネットワーク構造の定義時**: ネットワークを定義する際に、`nn.Dropout` 層を追加します。これは通常、全結合層や畳み込み層の後に配置されます。`nn.Dropout` の引数 `p` はドロップアウトする確率を表し、一般的には `0.5` が使われますが、タスクやネットワーク構造に応じて調整することがあります。
2. **順伝播関数内**: 順伝播（forward）関数内でドロップアウト層を適用します。通常は活性化関数（例：ReLU）の後に適用されます。

#### 注意点
ドロップアウトは訓練時にのみ使用され、評価（テスト）時には使用されません。PyTorchでは、`model.train()` と `model.eval()` を使ってモデルのモードを切り替え、ドロップアウト層を制御します。`model.eval()` が呼び出されるとドロップアウト層は無効になります。

---

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 128)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 248)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(248, 128)
        self.dropout3 = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(128, 64)
        self.dropout4 = nn.Dropout(p=0.5)
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = F.relu(self.fc4(x))
        x = self.dropout4(x)
        x = self.fc5(x)
        return x

net = Net()


### L2正則化

L2正則化は、モデルの過学習を抑制するために用いられる一般的な手法です。過学習は、モデルが訓練データに対して過度に最適化され、新しいデータに対してうまく機能しない状態を指します。L2正則化は、この問題に対処するためにモデルのパラメータに制約を加える方法です。

#### 概要
- **原理**: L2正則化では、モデルの重みの二乗和（L2ノルム）に比例するペナルティを損失関数に追加します。これにより、重みが大きくなりすぎることを防ぎ、モデルが訓練データのノイズに過敏に反応するのを抑制します。
- **効果**: 重みが大きくなりすぎると、モデルが訓練データの特定の特徴やノイズに対して過剰に反応する可能性があります。L2正則化を使用することで、モデルの重みが小さく保たれ、より汎化されたモデルを作ることができます。

#### PyTorchでの実装
PyTorchでは、オプティマイザの設定を通じてL2正則化を簡単に適用することができます。例えば、`torch.optim` パッケージ内のオプティマイザに `weight_decay` パラメータを設定することで、L2正則化をモデルに適用することができます。



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()



criterion = nn.CrossEntropyLoss()
# L2正則化の適用（weight_decayパラメータで指定）
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
